In [1]:
library(survival)
library(SurvRegCensCov)

**(1) The data of time to death ($T_1,...,T_n$) are given as follows:**

Steroid: 1, 1+(3), 4+, 5, 7, 8, 10, 10+, 12+, 16+(3)
Control: 1+, 2(3), 3(2), 3+, 5+(2), 16+(6)

In [2]:
data = data.frame(
    time   = c(1, rep(1, 3), 4, 5, 7, 8, 10, 10, 12, rep(16, 3), 1, rep(2, 3), 3, 3, 3, 5, 5, rep(16, 6)),
    status = c(1, rep(0, 3), 0, 1, 1, 1,  1,  0,  0, rep( 0, 3), 0, rep(1, 3), 1, 1, 0, 0, 0, rep(0, 6)),
    treat = c(rep(1, 14), rep(0, 15))
)

Surv(data$time, data$status)

 [1]  1   1+  1+  1+  4+  5   7   8  10  10+ 12+ 16+ 16+ 16+  1+  2   2   2   3 
[20]  3   3+  5+  5+ 16+ 16+ 16+ 16+ 16+ 16+

**(a) This homework is based on Weibull distribution assumption as follows. Let $Z=1$ for the steroid and $Z = 0$ for the control. Assume that survival time $T|Z$ is Weibull with scale parameter $\lambda_Z$ and shape parameter $\gamma$ (exponential if $\gamma = 1$) and the survival function is**

$$
S_Z(t) = exp\{-\lambda_Z t^{\gamma}\}
$$
**where**
$$
\lambda_Z = \frac{1}{\exp\{(\beta_0 + \beta_1 Z)\gamma\}} = \exp\{(-\beta_0 - \beta_1 Z)\gamma\}
$$


**Write down the corresponding hazard function $h_Z(t)$**

We can get the CDF,

$$
F_Z(t) = 1 - S(t)
$$

$$
F_Z(t) = 1 - \exp(-\lambda_Z t^{\gamma})
$$

Differentiate for the pdf,

$$
f_Z(t) = {d\over dt}F(t)
$$

$$
f_Z(t) = {d\over dt}(1 - \exp(-\lambda_Z t^{\gamma}))
$$

$$
f_Z(t) = - \exp(-\lambda_Z t^{\gamma})(-\lambda_Z \gamma t^{\gamma - 1})
$$

$$
f_Z(t) = \lambda_Z \gamma t^{\gamma - 1}\exp(-\lambda_Z t^{\gamma})
$$

The hazard function is given by,

$$
h_Z(t) = {f(t) \over S(t)}
$$

$$
h_Z(t) = {\lambda_Z \gamma t^{\gamma - 1}\exp(-\lambda_Z t^{\gamma}) \over exp\{-\lambda_Z t^{\gamma}\}}
$$

$$
h_Z(t) = \lambda_Z \gamma t^{\gamma - 1}
$$

**(b) Write down the likelihood function on $\beta_0$, $\beta_1$ and $\gamma$;  Explain briefly how you might find the maximum-likelihood estimates of $\beta_0$, $\beta_1$ and $\gamma$ (do not attempt to find them)**

We have that the $T_1...T_n$ denote the time associated with each unit. Let $C_i$ be an indicator variable for whether the ith unit is censored. Let $Z_i$ be an indicator variable for whether the ith unit is in the treatment group

Then, the likelihood is given by,

$$
L(\lambda_Z, \gamma) = \prod_{i = n}^n S_{Z_i}(T_i)^{C_i}f_{Z_i}(T_i)^{1 - C_i}
$$

Substitute for $f$ and $S$.

$$
f_Z(t) = \lambda_Z \gamma t^{\gamma - 1}\exp(-\lambda_Z t^{\gamma})
$$

$$
S_Z(t) = \exp(-\lambda_Z t^{\gamma})
$$

$$
L(\lambda_Z, \gamma) = \prod_{i = n}^n (\exp(-\lambda_{Z_i} T_i^{\gamma}))^{C_i} (\lambda_{Z_i} \gamma T_i^{\gamma - 1}\exp(-\lambda_{Z_i} T_i^{\gamma}))^{1 - C_i}
$$

Simplify,

$$
L(\lambda_Z, \gamma) = \prod_{i = n}^n\exp(-\lambda_{Z_i} T_i^{\gamma})(\lambda_{Z_i} \gamma T_i^{\gamma - 1})^{1 - C_i}
$$

We have that,

$$
\lambda_Z = \exp((-\beta_0 - \beta_1 Z)\gamma)
$$

So,

$$
L(\beta_0, \beta_1, \gamma) = \prod_{i = n}^n\exp(-\exp((-\beta_0 - \beta_1 Z_i)\gamma) T_i^{\gamma})(\exp((-\beta_0 - \beta_1 Z_i)\gamma) \gamma T_i^{\gamma - 1})^{1 - C_i}
$$

To get the maximum likelihood estimates for $\beta_0$, $\beta_1$, $\gamma$, we would log this, and take the derivative. Then, we would set it to zero and solve for the parameters in terms of the data. Alternatively, we could fix the data and use gradient descent to fit the parameters.

**(c) Find estimates of $\beta_0$, $\beta_1$ and $\gamma$ using proc 'lifereg' in SAS (or 'survreg' in R). What is the p-value on $\beta_1$? Please state the null and alternative hypotheses. Is there a big difference on p-value, in comparison with log-rank test in (a)?**

In [3]:
log_rank <- survdiff(Surv(time, status) ~ treat, data = data)
log_rank

Call:
survdiff(formula = Surv(time, status) ~ treat, data = data)

         N Observed Expected (O-E)^2/E (O-E)^2/V
treat=0 15        5     5.18   0.00596    0.0129
treat=1 14        5     4.82   0.00639    0.0129

 Chisq= 0  on 1 degrees of freedom, p= 0.9 

The log rank test is testing $H_0: S(t | T = 0) = S(t | T = 1)$ vs $H_1: S(t | T = 0) = S(t | T = 1)$ under the assumption that $S(t | T = 0) = cS(t | T = 1)$ for some $c \in \mathbb{R}$. It gets a $p = 0.9$, we we can't detect a different in the survival functions between treatment and control.

In [4]:
fit <- survreg(Surv(time, status) ~ treat, dist = "weibull", data = data)
summary(fit)


Call:
survreg(formula = Surv(time, status) ~ treat, data = data, dist = "weibull")
             Value Std. Error     z       p
(Intercept)  3.287      0.556  5.91 3.4e-09
treat       -0.127      0.706 -0.18    0.86
Log(scale)   0.110      0.269  0.41    0.68

Scale= 1.12 

Weibull distribution
Loglik(model)= -41.2   Loglik(intercept only)= -41.3
	Chisq= 0.03 on 1 degrees of freedom, p= 0.86 
Number of Newton-Raphson Iterations: 5 
n= 29 


The relevant p-value here is the one associated with the treat parameter. This is testing $H_0: \lambda_0 = \lambda_1$ vs $H_1: \lambda_0 = \lambda_1$ under the assuption that the the data are Weibull distributed, parametrized by lambda as in (a). We find $p = 0.86$. This is similar to the p-value for the log rank test.

**(d) What is the predicted median survival time for each group under the Weibull assumption?**

In [5]:
predict(fit, newdata = data.frame(treat = c(0, 1)), type = "quantile", p = 0.5)

1        2 
17.77276 15.66019

So, the predicted median survival time is 17.77 in control and 15.66 in treatment.

**(e) What is hazard ratio of treatment effect and its 95\% CI (you may use the R package/library `SurvRegCensCov') ?**

In [6]:
converted_fit = ConvertWeibull(fit, conf.level = 0.95)
converted_fit

$vars
         Estimate         SE
lambda 0.05267814 0.03826071
gamma  0.89553169 0.24086577
treat  0.11332507 0.63271976

$HR
            HR        LB       UB
treat 1.119996 0.3240742 3.870691

$ETR
            ETR        LB      UB
treat 0.8811345 0.2206899 3.51805

So, the hazard ratio MLE is 1.119996, indicating a higher hazard rate for treatment. However, the 95% confidence interval is $[0.3240742, 3.870691]$, which includes values smaller and larger than 1. So, the result is not statistically significant.

**(f) A survival model is considered as an accelerated failure time (AFT) model if $S(t | Z=1) = S(t/\phi | Z = 0)$ where $\phi^{-1}$ is accelerated factor (a constant). Is this Weibull model a AFT? If yes, what is the value of the accelerated factor $\phi^{-1}$ (hint: review course notes Page 137, 143 and 146).**

Yes, it is an AFT. The acceleration factor is given by,

$$
\phi^{-1} = \exp(\beta_1/ \gamma)
$$

In [7]:
beta_hat = converted_fit$vars["treat", "Estimate"]
gamma_hat = converted_fit$vars["gamma", "Estimate"]
phi_hat = exp(beta_hat / gamma_hat)

phi_hat

[1] 1.134901

The acceleration factor is 1.135. The loose interpretation here is that units in treatment age 1.135 times faster than units in control. More exactly, the survival function for treatment units is the same as for control units which are 1.135 times older.

**(g) Estimate the median follow-up time for each group using reverse Kaplan-Meier estimation.**

In [8]:
data$rev_status = 1 - data$status

rev_km0 = survfit(Surv(time, rev_status) ~ 1, data=subset(data, treat==0))
rev_km1 = survfit(Surv(time, rev_status) ~ 1, data=subset(data, treat==1))

summary(rev_km0)
summary(rev_km1)

Call: survfit(formula = Surv(time, rev_status) ~ 1, data = subset(data, 
    treat == 0))

 time n.risk n.event survival std.err lower 95% CI upper 95% CI
    1     15       1    0.933  0.0644        0.815            1
    3     11       1    0.848  0.0999        0.674            1
    5      8       2    0.636  0.1499        0.401            1
   16      6       6    0.000     NaN           NA           NA

Call: survfit(formula = Surv(time, rev_status) ~ 1, data = subset(data, 
    treat == 1))

 time n.risk n.event survival std.err lower 95% CI upper 95% CI
    1     14       3    0.786   0.110        0.598        1.000
    4     10       1    0.707   0.124        0.502        0.996
   10      6       1    0.589   0.149        0.359        0.967
   12      4       1    0.442   0.170        0.208        0.938
   16      3       3    0.000     NaN           NA           NA

In [9]:
summary(rev_km0)$table["median"]
summary(rev_km1)$table["median"]

median 
    16

median 
    12

So, for the control group, median follow up time is 16. For the treatment group, it's 12.